# Visualization Workflow

## Install packages

In [1]:
print("Installing packages (this could take awhile)...")

import sys
!{sys.executable} -m pip install -r visualization_requirements.txt --quiet --exists-action i
print("Done!")

import os
import pathlib
from typing import Optional, Dict
import time
import warnings
from functools import partial
from utils import *
import filepattern


Installing packages (this could take awhile)...
Done!


## Inputs

In [5]:
inpDir = pathlib.Path('/opt/shared/notebooks/eastman/outputs/labels')
outDir = pathlib.Path('/opt/shared/notebooks/eastman')
platesNum=200
dryrun=False


### Running All functions Sequentially

In [7]:
warnings.filterwarnings('ignore')

def visualizations_workflow(inpDir:pathlib.Path,
                    outDir: pathlib.Path,
                    platesNum:int,
                    dryrun:bool=True         
                   ):
    
    # os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    starttime= time.time() 
        
    plates = [str(i).zfill(3) for i in range(1, platesNum+1)]


    assert platesNum != 0, "Number of Processing Plates should be greater than zero"

    filePatterns = ['p'+ pl +  '_x{x+}_y{y+}_wx{t}_wy{p}_c2.ome.tif'  for pl in plates]

    assert len(filePatterns) == platesNum, "Number of filepattern should be equivalent to number of plates"



    for plate, filePattern in zip(plates, filePatterns):
        
        
        wstime= time.time() 
        
        starttime= time.time() 
        
        print('Step 1: Converting labelled images to binary')
    
        outpath = convert_label_to_binary(
                    inpDir=inpDir,
                    outDir=outDir,
                    pattern=filePattern,
                    dryrun=dryrun)

        
        endtime = round((time.time() - starttime)/60, 2)
        print(f'Step 1: Completed in {endtime} minutes')

        print(f'Workflow runs for plate:{plate} & filePattern:{filePattern}')
        
        print(f'Step 2: montage on plate: p{plate}')
        
        starttime= time.time() 

        stitchpath = montage(inpDir=outpath,
                           outDir=outDir,
                           filePattern=filePattern,
                           dryrun=dryrun) 

        endtime = round((time.time() - starttime)/60, 2)
        print(f'Step 2 completed in: {endtime} minutes')

       
        print(f'Step 3: assemble on plate: p{plate}')


        starttime= time.time() 
        asspath = assemble(inpDir=outpath,
                    outDir=outDir,
                    stitchPath=stitchpath,
                    filePattern=filePattern,
                    dryrun=dryrun)

        endtime = round((time.time() - starttime)/60, 2)
        print(f'Step 3 completed in: {endtime} minutes')

        print(f'Step 5: precomputeslide on plate: p{plate}')

        starttime= time.time() 
    
        outpre = pathlib.Path('/home/jovyan/shared/data/file-server/pyramids/eastmannew/zarrpyramids_segmentations')
                               

        precompath = precompute_slide(inpDir=asspath,
                           outDir=outpre,
                           plate=plate,
                           imageType='segmentation',
                           pyramidType="Zarr",
                           dryrun=dryrun)

        endtime = round((time.time() - starttime)/60, 2)
        print(f'Step 5: complete in {endtime} minutes')

        pftime = round((time.time() - wstime)/60, 2)
        
        print(f'Runtime for visualization workflow for Segmentations: p{plate} is {pftime}')
        

    return


visualizations_workflow(inpDir,
                    outDir,
                    platesNum,
                    dryrun=False
                
                   )    